In [2]:
from collections import defaultdict, deque

# Directed graph
class DiGraph:
    def __init__(self, n):
        self.n = n
        self.adj = [[] for _ in range(n)]

    def add_edge(self, u, v):
        self.adj[u].append(v)

    def reverse(self):
        r = DiGraph(self.n)
        for u in range(self.n):
            for v in self.adj[u]:
                r.add_edge(v, u)
        return r


# Problem 1: SCC (Kosaraju)
class SCCResult:
    def __init__(self):
        self.compCount = 0
        self.compId = []
        self.comps = []
        self.dag = None


def dfs1(g, v, used, order):
    used[v] = True
    for to in g.adj[v]:
        if not used[to]:
            dfs1(g, to, used, order)
    order.append(v)


def dfs2(g, v, used, compId, cid, cur):
    used[v] = True
    compId[v] = cid
    cur.append(v)
    for to in g.adj[v]:
        if not used[to]:
            dfs2(g, to, used, compId, cid, cur)


def scc_kosaraju(g):
    n = g.n
    used = [False] * n
    order = []

    for i in range(n):
        if not used[i]:
            dfs1(g, i, used, order)

    gr = g.reverse()
    used = [False] * n
    compId = [-1] * n
    order.reverse()

    comps = []
    cid = 0
    for v in order:
        if not used[v]:
            cur = []
            dfs2(gr, v, used, compId, cid, cur)
            comps.append(cur)
            cid += 1

    dag = DiGraph(cid)
    seen = set()
    for u in range(n):
        for v in g.adj[u]:
            cu, cv = compId[u], compId[v]
            if cu != cv:
                key = (cu, cv)
                if key not in seen:
                    seen.add(key)
                    dag.add_edge(cu, cv)

    res = SCCResult()
    res.compCount = cid
    res.compId = compId
    res.comps = comps
    res.dag = dag
    return res


# Problem 2: Euler tour (directed, Hierholzer)
# Includes strong connectivity check for vertices with non-zero degree

def euler_tour(g, start):
    n = g.n
    indeg = [0] * n
    outdeg = [0] * n

    for u in range(n):
        outdeg[u] = len(g.adj[u])
        for v in g.adj[u]:
            indeg[v] += 1

    # Degree condition
    for i in range(n):
        if indeg[i] != outdeg[i]:
            return None

    # Connectivity check: all vertices with edges must be in one SCC
    scc = scc_kosaraju(g)
    active_comp = None
    for i in range(n):
        if indeg[i] + outdeg[i] > 0:
            if active_comp is None:
                active_comp = scc.compId[i]
            elif scc.compId[i] != active_comp:
                return None

    # Hierholzer algorithm
    work = [deque(g.adj[i]) for i in range(n)]
    st = [start]
    ans = []

    while st:
        v = st[-1]
        if work[v]:
            to = work[v].popleft()
            st.append(to)
        else:
            ans.append(st.pop())

    ans.reverse()
    return ans



# Problem 3: Topological order (Kahn)

def topo_order(g, prefer_first):
    V = []
    seen = set()
    for u in g:
        if u not in seen:
            V.append(u)
            seen.add(u)
        for v in g[u]:
            if v not in seen:
                V.append(v)
                seen.add(v)

    indeg = {v: 0 for v in V}
    for u in g:
        for v in g[u]:
            indeg[v] += 1

    zeros = [v for v in V if indeg[v] == 0]
    q = deque()
    if prefer_first in zeros:
        zeros.remove(prefer_first)
        q.append(prefer_first)
    for z in zeros:
        q.append(z)

    order = []
    while q:
        u = q.popleft()
        order.append(u)
        for v in g.get(u, []):
            indeg[v] -= 1
            if indeg[v] == 0:
                q.append(v)

    if len(order) != len(V):
        return None
    return order


# Utility printing

def print_int_graph(g, title):
    print(title)
    for u in range(g.n):
        print(f"  {u} -> {g.adj[u]}")
    print()


def print_dag(dag, title):
    print(title)
    for c in range(dag.n):
        outs = [f"C{to}" for to in dag.adj[c]]
        print(f"  C{c} -> {outs}")
    print()


if __name__ == "__main__":
    # Problem 1
    g1 = DiGraph(5)
    g1.add_edge(0, 1)
    g1.add_edge(1, 2)
    g1.add_edge(2, 0)
    g1.add_edge(2, 3)
    g1.add_edge(3, 4)
    g1.add_edge(4, 3)

    print_int_graph(g1, "Problem 1: G")
    print_int_graph(g1.reverse(), "Problem 1: reverse(G)")

    scc = scc_kosaraju(g1)
    print("Problem 1: SCCs")
    for i, comp in enumerate(scc.comps):
        print(f"  C{i} = {comp}")
    print()
    print_dag(scc.dag, "Problem 1: SCC graph")

    # Problem 2
    eg = DiGraph(3)
    eg.add_edge(0, 1)
    eg.add_edge(1, 2)
    eg.add_edge(2, 0)
    eg.add_edge(0, 2)
    eg.add_edge(2, 1)
    eg.add_edge(1, 0)

    print_int_graph(eg, "Problem 2: Euler graph")
    print("Problem 2: Euler tour =", euler_tour(eg, 0))
    print()

    # Problem 3
    courses = {
        "A": ["B", "C"],
        "B": ["C", "D"],
        "C": ["E"],
        "D": ["E", "F"],
        "G": ["F", "E"],
    }

    print("Problem 3: topo (prefer A) =", topo_order(courses, "A"))
    print("Problem 3: topo (prefer G) =", topo_order(courses, "G"))


Problem 1: G
  0 -> [1]
  1 -> [2]
  2 -> [0, 3]
  3 -> [4]
  4 -> [3]

Problem 1: reverse(G)
  0 -> [2]
  1 -> [0]
  2 -> [1]
  3 -> [2, 4]
  4 -> [3]

Problem 1: SCCs
  C0 = [0, 2, 1]
  C1 = [3, 4]

Problem 1: SCC graph
  C0 -> ['C1']
  C1 -> []

Problem 2: Euler graph
  0 -> [1, 2]
  1 -> [2, 0]
  2 -> [0, 1]

Problem 2: Euler tour = [0, 1, 2, 0, 2, 1, 0]

Problem 3: topo (prefer A) = ['A', 'G', 'B', 'C', 'D', 'E', 'F']
Problem 3: topo (prefer G) = ['G', 'A', 'B', 'C', 'D', 'E', 'F']
